# `ProteinWorkshop` Tutorial, Part 1 - Training a New Model
![Models](../docs/source/_static/box_models.png)

## Train a new model using the `ProteinWorkshop`

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

Welcome to the tutorial series for the `ProteinWorkshop`! 

In the `ProteinWorkshop`, we implement numerous [featurisation](https://www.proteins.sh/configs/features) schemes, [datasets](https://www.proteins.sh/configs/dataset) for [self-supervised pre-training](https://proteins.sh/quickstart_component/pretrain.html) and [downstream evaluation](https://proteins.sh/quickstart_component/downstream.html), [pre-training](https://proteins.sh/configs/task) tasks, and [auxiliary tasks](https://proteins.sh/configs/task.html#auxiliary-tasks).

[Processed datasets](https://drive.google.com/drive/folders/18i8rLST6ZICTBu6Q67ClT0KqN9AHeqoW?usp=sharing) and [pre-trained weights](https://drive.google.com/drive/folders/1zK1r8FpmGaqV_QwUJuvDacwSL0RW-Vw9?usp=sharing) are made available. Downloading datasets is not required; upon first run all datasets will be downloaded and processed from their respective source.

The `ProteinWorkshop` encompasses several models as well as pre-trained weights for them so that you can readily use them.

In this tutorial, we show you how you can use what is already available in the protein workshop to train and use models for specific tasks. The `ProteinWorkshop` is structured as a very modular package; we will therefore talk about how to change the different parts of it, like the model, training task, dataset, featurization scheme, etc. in this tutorial. 

Besides using all the different options we provide, you can make use of the modular nature of the `ProteinWorkshop` to add your own models, datasets, featurization schemes, and training tasks. We will show you how to do this in the next tutorials.

To train a new model, you can follow the following 3-step procedure:

1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks
2. Validate the designed training config
3. Use the designed config to train a new model

### 1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks

You can switch out any of these for another available option by replacing the corresponding argument's value in `overrides`:

`cfg = hydra.compose("template", overrides=["encoder=schnet", "task=inverse_folding", "dataset=afdb_swissprot_v4", "features=ca_base", "+aux_task=none"], return_hydra_config=True)`

In [2]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")
# print(rel_path)
GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=dimenet_plus_plus",
        "encoder.hidden_channels=512",  # Number of channels in the hidden layers
        "encoder.out_dim=32", # Output dimension of the model
        "encoder.num_layers=6",  # Number of layers in the model
        "encoder.int_emb_size=64",  # Embedding size for interaction features
        "encoder.basis_emb_size=8",  # Embedding size for basis functions
        "encoder.out_emb_channels=256",  # Number of channels in the output embeddings
        "encoder.num_spherical=7",  # Number of spherical harmonics
        "encoder.num_radial=6",  # Number of radial basis functions
        "encoder.cutoff=10.0",  # Cutoff distance for interactions
        "encoder.max_num_neighbors=32", # Maximum number of neighboring atoms to consider
        "encoder.envelope_exponent=5",  # Exponent of the envelope function
        "encoder.num_before_skip=1",  # Number of layers before the skip connections
        "encoder.num_after_skip=2",  # Number of layers after the skip connections
        "encoder.num_output_layers=3",  # Number of output layers
        "encoder.act=swish",  # Activation function to use
        "encoder.readout=add",  # Global pooling method to be used
        "encoder.pretraining=True",

        # "decoder.graph_label.dummy=True",
        "decoder=subgraph_distances", # here
        "decoder.subgraph_distances.hidden_channels=32", # here

        "task=subgraph_distance_prediction", # here
        "dataset=afdb_swissprot_v4", # here
        "dataset.datamodule.batch_size=32", # here
        "dataset.datamodule.train_split=0.5", # here
        "dataset.datamodule.val_split=0.05", # here
        "features=fe_subgraph",  # here
        "+aux_task=none",
        
        "trainer.max_epochs=1000",
        "optimiser=adam",
        "optimiser.optimizer.lr=5e-4",
        "callbacks.early_stopping.patience=20", # to change, don't comment this
        "test=True",
        "scheduler=steplr", # to set parameters in yaml
        # "optimizer.weight_decay=0.5"
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[07/18/24 18:47:27] DEBUG    No `.env` file found in project root. Checking for env vars...         ]8;id=45778;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=850906;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#22\22]8;;\

                    DEBUG    No env var `DATA_PATH` found. Setting default...                       ]8;id=803477;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=479537;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#28\28]8;;\

                    INFO     DATA_PATH:                                                             ]8;id=213172;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=840517;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#39\39]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/data                          

[07/18/24 18:47:29] INFO     Hydra initialised at                                                    ]8;id=58799;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=117786;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/config.                       

### 2. Validate the designed training config

This is not strictly necessary, but it is a good idea to validate the config before training. This will check that all the arguments you have provided are valid and that the config is complete.

In [3]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

                    DEBUG    CUDA available: True                                                     ]8;id=721956;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=996242;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=695526;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=243403;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=646501;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=139725;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=266543;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=119844;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

In [4]:
print(cfg.keys())
for key in cfg.keys():
    print(key)
    print(cfg[key])

dict_keys(['hydra', 'env', 'dataset', 'features', 'encoder', 'decoder', 'transforms', 'callbacks', 'optimiser', 'scheduler', 'trainer', 'extras', 'metrics', 'task', 'logger', 'name', 'seed', 'num_workers', 'task_name', 'test', 'aux_task'])
hydra
{'run': {'dir': '${env.paths.log_dir}/${task_name}/runs/${now:%Y-%m-%d}_${now:%H-%M-%S}'}, 'sweep': {'dir': '${env.paths.log_dir}/${task_name}/multiruns/${now:%Y-%m-%d}_${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from those groups (group=option)\n\n$

### 3. Use the designed config to train a new model

Now with the config you have designed, you can train a new model. You can also use the `ProteinWorkshop` to evaluate the model on a downstream task.

In training:

ProteinBatch(fill_value=[32], atom_list=[32], coords=[15602, 37, 3], residues=[32], residue_id=[32], 
chains=[15602], residue_type=[15602], b_factor=[15602], id=[32], x=[15602, 23], seq_pos=[15602, 1], batch=[15602], 
ptr=[33], pos=[15602, 3], edge_index=[2, 247543], subgraphs=[1548, 149], subgraph_distances=[1548], 
subgraph_lengths=[1548])


In [5]:
from proteinworkshop.train import train_model

train_model(cfg)

Seed set to 52


[07/18/24 18:47:30] INFO     Instantiating datamodule:                                                  ]8;id=896664;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=987468;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#99\99]8;;\
                             <graphein.ml.datasets.foldcomp_dataset.FoldCompLightningDataModule...                 

100%|██████████| 542378/542378 [00:00<00:00, 4697883.10it/s]
Processing...
Done!


[07/18/24 18:47:31] INFO     Instantiating callbacks...                                                ]8;id=990167;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=446377;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#106\106]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.ModelCheckpoint>   ]8;id=440939;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=690437;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

...

[07/18/24 18:55:06] WARNING  Ran out of memory in the forward pass. Skipping current train batch with   ]8;id=218259;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=591075;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#648\648]8;;\
                             index 624.                                                                            

                    WARNING  Ran out of memory in the forward pass. Skipping current train batch with   ]8;id=943361;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=212012;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#648\648]8;;\
                             index 625.                                                                            

[07/18/24 18:55:07] WARNING  Ran out of memory in the forward pass. Skipping current train batch with   ]8;id=813924;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=519998;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#648\648]8;;\
                             index 626.                                                                            

                    WARNING  Ran out of memory in the forward pass. Skipping current train batch with   ]8;id=18417;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=908748;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#648\648]8;;\
                             index 627.                                                                            

[07/18/24 18:55:08] WARNING  Ran out of memory in the forward pass. Skipping current train batch with   ]8;id=465415;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py\base.py]8;;\:]8;id=119345;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/models/base.py#648\648]8;;\
                             index 628.                                                                            

### 4. Wrapping up

Have any additional questions about using the components provided in the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to leverage the full power of the repository!

/home/yang/anaconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py